In [ ]:
 mö.# 1) Gerekirse eski klonları temizle (opsiyonel)
!rm -rf YapayZekaChatbot

# 2) Senin GitHub repoyu klonla
!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

# 3) Klasöre gir
%cd YapayZekaChatbot/bilgi_guvenligi_dataset

# 4) İçerikleri listele
!ls


Cloning into 'YapayZekaChatbot'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 100 (delta 52), reused 90 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (100/100), 23.98 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/YapayZekaChatbot/bilgi_guvenligi_dataset
ISO-27001-Controls-List-Free-Download-pdf-.pdf
SANS_Acceptable_Encryption_Standard_April2025.pdf
SANS_Acceptable_Use_Standard_April2025.pdf
SANS_Access_Management_Policy_April2025.pdf
SANS_Acquisition_Assessment_Standard_April2025.pdf
SANS_Artificial_Intelligence_Standard_April2025.pdf
SANS_Asset_Management_Policy_April2025.pdf
SANS_Cloud_Service_Provider_Management_Policy_April2025.pdf
SANS_Configuration_Management_Policy_April2025.pdf
SANS_Database_Credentials_Standard_April2025.pdf
SANS_Data_Inventory_Management_Policy_April2025.pdf
SANS_Education_Management_Policy_April2025.pdf
SANS_Email

In [ ]:
!pip install pypdf python-docx pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.1 MB/s eta 0:00:00


In [ ]:
import os
from pypdf import PdfReader
import docx
import pandas as pd
from tqdm.auto import tqdm

folder = "."  # şu an zaten bilgi_guvenligi_dataset içindeyiz

rows = []

def read_pdf(path):
    reader = PdfReader(path)
    texts = []
    for i, page in enumerate(reader.pages):
        try:
            text = page.extract_text() or ""
        except Exception:
            text = ""
        texts.append((i, text))
    return texts

def read_docx(path):
    doc = docx.Document(path)
    # Word için sayfa bilgisi yok, paragraf bazlı okuyacağız
    full_text = []
    for para in doc.paragraphs:
        if para.text.strip():
            full_text.append(para.text.strip())
    joined = "\n".join(full_text)
    # tek "page = 0" gibi düşünebiliriz
    return [(0, joined)]

# klasördeki tüm dosyaları gez
files = [f for f in os.listdir(folder) if f.lower().endswith((".pdf", ".docx"))]

for fname in tqdm(files):
    path = os.path.join(folder, fname)
    if fname.lower().endswith(".pdf"):
        pages = read_pdf(path)
    else:
        pages = read_docx(path)

    for page_num, text in pages:
        text = text.strip()
        if not text:
            continue
        rows.append({
            "filename": fname,
            "page_or_section": page_num,
            "text": text
        })

df = pd.DataFrame(rows)
len(df), df.head()


  0%|          | 0/37 [00:00<?, ?it/s]

(152,
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                1   
 2  SANS_Software_Development_Vulnerability_Manage...                2   
 3  SANS_Software_Development_Vulnerability_Manage...                3   
 4         SANS_Acceptable_Use_Standard_April2025.pdf                0   
 
                                                 text  
 0  Page\t1 \nSoftware Development Vulnerability M...  
 1  Page\t2 \nSafeguards To achieve the organizati...  
 2  Page\t3 \nSDV-09 Ensure that each of the appro...  
 3  Page\t4 \nPolicy Sanctions Non-compliance with...  
 4  Page\t1 \nAcceptable Use Standard (Last Update...  )

In [ ]:
df.to_csv("bilgi_guvenligi_raw_text.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_raw_text.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re

def filename_to_url(filename):
    # .pdf veya .docx uzantısını at
    name = re.sub(r"\.(pdf|docx)$", "", filename, flags=re.IGNORECASE)

    # SANS_ prefiksini at
    name = name.replace("SANS_", "")

    # Sonundaki tarih kısmını sil (ör: _April2025)
    name = re.sub(r"_April20\d\d", "", name)

    # Alt çizgi → tire
    name = name.replace("_", "-")

    # Küçük harfe çevir
    name = name.lower()

    # URL oluştur
    return f"https://www.sans.org/information-security-policy/{name}/"


In [ ]:
ISO_URL = "https://hightable.io/iso-27001-annex-a-controls-list/"  # ISO 27001:2022 Annex A Controls


In [ ]:
df["source_url"] = df["filename"].apply(
    lambda x: ISO_URL if "ISO-27001" in x else filename_to_url(x)
)

df[["filename", "source_url"]].head(10)


,filename,source_url
0,SANS_Software_Development_Vulnerability_Manage...,https://www.sans.org/information-security-poli...
1,SANS_Software_Development_Vulnerability_Manage...,https://www.sans.org/information-security-poli...
2,SANS_Software_Development_Vulnerability_Manage...,https://www.sans.org/information-security-poli...
3,SANS_Software_Development_Vulnerability_Manage...,https://www.sans.org/information-security-poli...
4,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...
5,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...
6,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...
7,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...
8,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...
9,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...


In [ ]:
df[df["filename"].str.contains("ISO-27001", case=False)][["filename", "page_or_section", "source_url"]]


,filename,page_or_section,source_url
53,ISO-27001-Controls-List-Free-Download-pdf-.pdf,0,https://hightable.io/iso-27001-annex-a-control...
54,ISO-27001-Controls-List-Free-Download-pdf-.pdf,1,https://hightable.io/iso-27001-annex-a-control...
55,ISO-27001-Controls-List-Free-Download-pdf-.pdf,2,https://hightable.io/iso-27001-annex-a-control...
56,ISO-27001-Controls-List-Free-Download-pdf-.pdf,3,https://hightable.io/iso-27001-annex-a-control...
57,ISO-27001-Controls-List-Free-Download-pdf-.pdf,4,https://hightable.io/iso-27001-annex-a-control...
58,ISO-27001-Controls-List-Free-Download-pdf-.pdf,5,https://hightable.io/iso-27001-annex-a-control...
59,ISO-27001-Controls-List-Free-Download-pdf-.pdf,6,https://hightable.io/iso-27001-annex-a-control...
60,ISO-27001-Controls-List-Free-Download-pdf-.pdf,7,https://hightable.io/iso-27001-annex-a-control...
61,ISO-27001-Controls-List-Free-Download-pdf-.pdf,8,https://hightable.io/iso-27001-annex-a-control...
62,ISO-27001-Controls-List-Free-Download-pdf-.pdf,9,https://hightable.io/iso-27001-annex-a-control...


In [ ]:
df.to_csv("bilgi_guvenligi_raw_text_with_source.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_raw_text_with_source.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_raw_text_with_source.csv"

df = pd.read_csv(url)

df.shape, df.head(3)


((152, 4),
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                1   
 2  SANS_Software_Development_Vulnerability_Manage...                2   
 
                                                 text  \
 0  Page\t1 \nSoftware Development Vulnerability M...   
 1  Page\t2 \nSafeguards To achieve the organizati...   
 2  Page\t3 \nSDV-09 Ensure that each of the appro...   
 
                                           source_url  
 0  https://www.sans.org/information-security-poli...  
 1  https://www.sans.org/information-security-poli...  
 2  https://www.sans.org/information-security-poli...  )

In [ ]:
import re

def clean_text(t: str) -> str:
    t = str(t).replace("\t", " ")
    lines = []
    for line in t.splitlines():
        s = line.strip()
        if not s:
            continue

        # Page 1 / Page 2 of 14 gibi satırları at
        if re.match(r"^Page\s+\d+(\s+of\s+\d+)?$", s):
            continue

        # ISO toolkit & classification çöpleri
        if "ULTIMATE ISO 27001 TOOLKIT" in s:
            continue
        if s.startswith("VIEW THE ULTIMATE ISO 27001"):
            continue
        if s.startswith("Classification: Confidential"):
            continue

        lines.append(s)

    return "\n".join(lines)

df["clean_text"] = df["text"].apply(clean_text)

# Boş kalanları at
df_clean = df[df["clean_text"].str.strip() != ""].copy()

df_clean.shape, df_clean.head(3)


((150, 5),
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                1   
 2  SANS_Software_Development_Vulnerability_Manage...                2   
 
                                                 text  \
 0  Page\t1 \nSoftware Development Vulnerability M...   
 1  Page\t2 \nSafeguards To achieve the organizati...   
 2  Page\t3 \nSDV-09 Ensure that each of the appro...   
 
                                           source_url  \
 0  https://www.sans.org/information-security-poli...   
 1  https://www.sans.org/information-security-poli...   
 2  https://www.sans.org/information-security-poli...   
 
                                           clean_text  
 0  Software Development Vulnerability Management ...  
 1  Safeguards To achieve the organization's overa...  
 2  SDV-09 Ensure that each of the approved softwa...  )

In [ ]:
df_clean.to_csv("bilgi_guvenligi_clean_text_with_source.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_clean_text_with_source.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

url_clean = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_clean_text_with_source.csv"

df_clean = pd.read_csv(url_clean)

df_clean.shape, df_clean.head(3)


((150, 5),
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                1   
 2  SANS_Software_Development_Vulnerability_Manage...                2   
 
                                                 text  \
 0  Page\t1 \nSoftware Development Vulnerability M...   
 1  Page\t2 \nSafeguards To achieve the organizati...   
 2  Page\t3 \nSDV-09 Ensure that each of the appro...   
 
                                           source_url  \
 0  https://www.sans.org/information-security-poli...   
 1  https://www.sans.org/information-security-poli...   
 2  https://www.sans.org/information-security-poli...   
 
                                           clean_text  
 0  Software Development Vulnerability Management ...  
 1  Safeguards To achieve the organization's overa...  
 2  SDV-09 Ensure that each of the approved softwa...  )

In [ ]:
# 30 karakterden kısa metinleri atalım (ör: "Purpose", "Scope" tek başına olanlar)
df_clean = df_clean[df_clean["clean_text"].str.len() > 30].copy()

df_clean.shape


(150, 5)

In [ ]:
def split_into_chunks(text, max_words=200, overlap=40):
    words = str(text).split()
    chunks = []
    start = 0
    n = len(words)
    if n == 0:
        return []

    while start < n:
        end = start + max_words
        chunk_words = words[start:end]
        chunks.append(" ".join(chunk_words))
        if end >= n:
            break
        start = end - overlap  # overlap kadar geri sar
    return chunks


In [ ]:
chunk_rows = []

for idx, row in df_clean.iterrows():
    chunks = split_into_chunks(row["clean_text"], max_words=200, overlap=40)
    for i, ch in enumerate(chunks):
        chunk_rows.append({
            "filename": row["filename"],
            "page_or_section": row["page_or_section"],
            "source_url": row["source_url"],
            "chunk_index": i,
            "chunk_text": ch
        })

df_chunks = pd.DataFrame(chunk_rows)

df_chunks.shape, df_chunks.head(3)


((252, 5),
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                0   
 2  SANS_Software_Development_Vulnerability_Manage...                1   
 
                                           source_url  chunk_index  \
 0  https://www.sans.org/information-security-poli...            0   
 1  https://www.sans.org/information-security-poli...            1   
 2  https://www.sans.org/information-security-poli...            0   
 
                                           chunk_text  
 0  Software Development Vulnerability Management ...  
 1  and individuals responsible for reviewing and ...  
 2  Safeguards To achieve the organization's overa...  )

In [ ]:
df_chunks.to_csv("bilgi_guvenligi_chunks.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_chunks.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks.csv"
df = pd.read_csv(url)

df.shape


(252, 5)

In [ ]:
iso_mask = df["filename"].str.contains("ISO-27001", case=False, na=False)


In [ ]:
df.loc[iso_mask, "chunk_text"] = (
    df.loc[iso_mask, "chunk_text"]
    .str.replace("Classification: Confidential", "", regex=False)
    .str.replace("Statement of Applicability", "", regex=False)
    .str.replace("ISO 27001 Annex A Controls List", "", regex=False)
    .str.strip()
)


In [ ]:
df[df["filename"].str.contains("ISO-27001")].head(10)


,filename,page_or_section,source_url,chunk_index,chunk_text
89,ISO-27001-Controls-List-Free-Download-pdf-.pdf,1,https://hightable.io/iso-27001-annex-a-control...,0,5 Organisational Controls 5.1 Policies for inf...
90,ISO-27001-Controls-List-Free-Download-pdf-.pdf,2,https://hightable.io/iso-27001-annex-a-control...,0,5.2 Information security roles and responsibil...
91,ISO-27001-Controls-List-Free-Download-pdf-.pdf,3,https://hightable.io/iso-27001-annex-a-control...,0,5.10 Acceptable use of information and other a...
92,ISO-27001-Controls-List-Free-Download-pdf-.pdf,3,https://hightable.io/iso-27001-annex-a-control...,1,and other associated assets should be establis...
93,ISO-27001-Controls-List-Free-Download-pdf-.pdf,4,https://hightable.io/iso-27001-annex-a-control...,0,5.18 Access rights Access rights to informatio...
94,ISO-27001-Controls-List-Free-Download-pdf-.pdf,4,https://hightable.io/iso-27001-annex-a-control...,1,"for acquisition, use, management and exit from..."
95,ISO-27001-Controls-List-Free-Download-pdf-.pdf,5,https://hightable.io/iso-27001-annex-a-control...,0,5.26 Response to information security incident...
96,ISO-27001-Controls-List-Free-Download-pdf-.pdf,6,https://hightable.io/iso-27001-annex-a-control...,0,5.34 Privacy and protection of PII The organiz...
97,ISO-27001-Controls-List-Free-Download-pdf-.pdf,6,https://hightable.io/iso-27001-annex-a-control...,1,the classification of the information to be ac...
98,ISO-27001-Controls-List-Free-Download-pdf-.pdf,7,https://hightable.io/iso-27001-annex-a-control...,0,6.4 Disciplinary process A disciplinary proces...


In [ ]:
df.to_csv("bilgi_guvenligi_chunks_clean_iso.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_chunks_clean_iso.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

url_clean = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_clean_text_with_source.csv"

df = pd.read_csv(url_clean)
df.shape, df.head(3)


((150, 5),
                                             filename  page_or_section  \
 0  SANS_Software_Development_Vulnerability_Manage...                0   
 1  SANS_Software_Development_Vulnerability_Manage...                1   
 2  SANS_Software_Development_Vulnerability_Manage...                2   
 
                                                 text  \
 0  Page\t1 \nSoftware Development Vulnerability M...   
 1  Page\t2 \nSafeguards To achieve the organizati...   
 2  Page\t3 \nSDV-09 Ensure that each of the appro...   
 
                                           source_url  \
 0  https://www.sans.org/information-security-poli...   
 1  https://www.sans.org/information-security-poli...   
 2  https://www.sans.org/information-security-poli...   
 
                                           clean_text  
 0  Software Development Vulnerability Management ...  
 1  Safeguards To achieve the organization's overa...  
 2  SDV-09 Ensure that each of the approved softwa...  )

In [ ]:
def normalize_text(t: str) -> str:
    t = str(t)
    # tab -> boşluk
    t = t.replace("\t", " ")
    # çoklu boşlukları tek boşluğa indir
    t = re.sub(r"[ ]{2,}", " ", t)
    # gereksiz boş satırları kaldır
    lines = [line.strip() for line in t.splitlines()]
    lines = [l for l in lines if l]
    # tekrar birleştir
    t = "\n".join(lines)
    return t.strip()

df["norm_text"] = df["clean_text"].apply(normalize_text)

# çok kısa anlamsız satırları at
df = df[df["norm_text"].str.len() > 30].copy()

df.shape


(150, 6)

In [ ]:
iso_mask = df["filename"].str.contains("ISO-27001", case=False, na=False)
df_iso = df[iso_mask].sort_values("page_or_section").copy()
df_sans = df[~iso_mask].copy()

len(df_iso), len(df_sans)


(13, 137)

In [ ]:
iso_text = "\n".join(df_iso["norm_text"].tolist())


In [ ]:
control_pattern = re.compile(r"(?m)^(?P<num>\d+\.\d+)\s+(?P<title>.+)")

controls = []
matches = list(control_pattern.finditer(iso_text))

for i, m in enumerate(matches):
    start = m.start()
    end = matches[i+1].start() if i+1 < len(matches) else len(iso_text)
    block = iso_text[start:end].strip()
    cid = m.group("num").strip()
    title = m.group("title").strip()
    controls.append({
        "control_id": cid,
        "control_title": title,
        "block_text": block
    })

iso_blocks_df = pd.DataFrame(controls)
iso_blocks_df["filename"] = df_iso["filename"].iloc[0]
iso_blocks_df["source_url"] = df_iso["source_url"].iloc[0]

iso_blocks_df.head()


,control_id,control_title,block_text,filename,source_url
0,5.1,Policies for information security,5.1 Policies for information security\nInforma...,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...
1,5.2,Information security roles and responsibilities,5.2 Information security roles and responsibil...,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...
2,5.3,Segregation of duties Conflicting duties and c...,5.3 Segregation of duties Conflicting duties a...,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...
3,5.4,Management Responsibilities,5.4 Management Responsibilities\nManagement sh...,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...
4,5.5,Contact with Authorities The organization shou...,5.5 Contact with Authorities The organization ...,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...


In [ ]:
section_heading_pattern = re.compile(
    r"(?m)^(Purpose|Scope|Safeguards|Policy Sanctions|Overview|Definitions|Standard)\b.*"
)

sans_rows = []

for fname, g in df_sans.groupby("filename"):
    # aynı policy'nin tüm sayfalarını sırayla birleştir
    g_sorted = g.sort_values("page_or_section")
    text = "\n".join(g_sorted["norm_text"].tolist())
    source_url = g_sorted["source_url"].iloc[0]

    matches = list(section_heading_pattern.finditer(text))
    sections = []

    if not matches:
        # hiç başlık yakalanmazsa, tüm policy'i tek section olarak ele al
        sections.append(("full", text))
    else:
        for i, m in enumerate(matches):
            sec_name = m.group(1)  # Purpose / Scope / Safeguards...
            start = m.start()
            end = matches[i+1].start() if i+1 < len(matches) else len(text)
            block = text[start:end].strip()
            sections.append((sec_name, block))

    for s_idx, (sec_name, block) in enumerate(sections):
        sans_rows.append({
            "filename": fname,
            "source_url": source_url,
            "section_label": sec_name,
            "section_index": s_idx,
            "block_text": block
        })

sans_blocks_df = pd.DataFrame(sans_rows)
sans_blocks_df.head()


,filename,source_url,section_label,section_index,block_text
0,SANS_Acceptable_Encryption_Standard_April2025.pdf,https://www.sans.org/information-security-poli...,full,0,Acceptable Encryption Standard (Last Updated A...
1,SANS_Acceptable_Use_Standard_April2025.pdf,https://www.sans.org/information-security-poli...,Safeguards,0,Safeguards General Use and Ownership <Company ...
2,SANS_Access_Management_Policy_April2025.pdf,https://www.sans.org/information-security-poli...,Safeguards,0,Safeguards To achieve the organization's overa...
3,SANS_Acquisition_Assessment_Standard_April2025...,https://www.sans.org/information-security-poli...,Policy Sanctions,0,Policy Sanctions Non-compliance with this poli...
4,SANS_Artificial_Intelligence_Standard_April202...,https://www.sans.org/information-security-poli...,full,0,Artificial Intelligence Standard (Last Updated...


In [ ]:
def split_into_chunks(text, max_words=180, overlap=40):
    words = str(text).split()
    chunks = []
    n = len(words)
    if n == 0:
        return []

    start = 0
    while start < n:
        end = start + max_words
        chunk_words = words[start:end]
        chunks.append(" ".join(chunk_words))
        if end >= n:
            break
        start = end - overlap  # overlap kadar geri sar
    return chunks


In [ ]:
iso_chunk_rows = []

for _, row in iso_blocks_df.iterrows():
    chunks = split_into_chunks(row["block_text"], max_words=180, overlap=40)
    for i, ch in enumerate(chunks):
        iso_chunk_rows.append({
            "filename": row["filename"],
            "source_url": row["source_url"],
            "section_label": f"ISO_{row['control_id']}",
            "section_index": row["control_id"],
            "chunk_index": i,
            "chunk_text": ch
        })

iso_chunks_df = pd.DataFrame(iso_chunk_rows)
iso_chunks_df.shape, iso_chunks_df.head(3)


((93, 6),
                                          filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...       ISO_5.2   
 2  https://hightable.io/iso-27001-annex-a-control...       ISO_5.3   
 
   section_index  chunk_index  \
 0           5.1            0   
 1           5.2            0   
 2           5.3            0   
 
                                           chunk_text  
 0  5.1 Policies for information security Informat...  
 1  5.2 Information security roles and responsibil...  
 2  5.3 Segregation of duties Conflicting duties a...  )

In [ ]:
sans_chunk_rows = []

for _, row in sans_blocks_df.iterrows():
    chunks = split_into_chunks(row["block_text"], max_words=180, overlap=40)
    for i, ch in enumerate(chunks):
        sans_chunk_rows.append({
            "filename": row["filename"],
            "source_url": row["source_url"],
            "section_label": row["section_label"],  # Purpose / Scope / Safeguards
            "section_index": row["section_index"],
            "chunk_index": i,
            "chunk_text": ch
        })

sans_chunks_df = pd.DataFrame(sans_chunk_rows)
sans_chunks_df.shape, sans_chunks_df.head(3)


((154, 6),
                                             filename  \
 0  SANS_Acceptable_Encryption_Standard_April2025.pdf   
 1  SANS_Acceptable_Encryption_Standard_April2025.pdf   
 2  SANS_Acceptable_Encryption_Standard_April2025.pdf   
 
                                           source_url section_label  \
 0  https://www.sans.org/information-security-poli...          full   
 1  https://www.sans.org/information-security-poli...          full   
 2  https://www.sans.org/information-security-poli...          full   
 
    section_index  chunk_index  \
 0              0            0   
 1              0            1   
 2              0            2   
 
                                           chunk_text  
 0  Acceptable Encryption Standard (Last Updated A...  
 1  Standard (AES) is strongly recommended for sym...  
 2  keys used to establish trust must be authentic...  )

In [ ]:
all_chunks = pd.concat([iso_chunks_df, sans_chunks_df], ignore_index=True)

# Çok kısa chunk'ları yine eleyelim (gürültü olmasın)
all_chunks = all_chunks[all_chunks["chunk_text"].str.len() > 40].copy()

all_chunks.shape, all_chunks.head(5)


((247, 6),
                                          filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 3  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 4  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...       ISO_5.2   
 2  https://hightable.io/iso-27001-annex-a-control...       ISO_5.3   
 3  https://hightable.io/iso-27001-annex-a-control...       ISO_5.4   
 4  https://hightable.io/iso-27001-annex-a-control...       ISO_5.5   
 
   section_index  chunk_index  \
 0           5.1            0   
 1           5.2            0   
 2           5.3            0   
 3           5.4            0   
 4           5.5            0   
 
                                   

In [ ]:
all_chunks.to_csv("bilgi_guvenligi_chunks_v2.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_chunks_v2.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

url_v2 = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v2.csv"
df = pd.read_csv(url_v2)
df.shape


(247, 6)

In [ ]:
iso_mask = df["filename"].str.contains("ISO-27001", case=False, na=False)
df[iso_mask].head(3)  # kontrol amaçlı


,filename,source_url,section_label,section_index,chunk_index,chunk_text
0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.1,5.1,0,5.1 Policies for information security Informat...
1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.2,5.2,0,5.2 Information security roles and responsibil...
2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.3,5.3,0,5.3 Segregation of duties Conflicting duties a...


In [ ]:
df.loc[iso_mask, "chunk_text"] = (
    df.loc[iso_mask, "chunk_text"]
      .str.replace("Classification: Confidential", "", regex=False)
      .str.replace("Statement of Applicability", "", regex=False)
      .str.replace("ISO 27001 Annex A Controls List", "", regex=False)
      .str.strip()
)


In [ ]:
# Hâlâ 'Classification: Confidential' geçen chunk kaldı mı?
(df["chunk_text"].str.contains("Classification: Confidential", na=False)).sum()


np.int64(0)

In [ ]:
df.to_csv("bilgi_guvenligi_chunks_v2_clean.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_chunks_v2_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

df = pd.read_csv("bilgi_guvenligi_chunks_v2_clean.csv")
df.shape


(247, 6)

In [ ]:
def normalize_text(t: str) -> str:
    t = str(t)
    t = t.replace("\t", " ")
    # birden fazla boşluğu teke indir
    t = re.sub(r"[ ]{2,}", " ", t)
    # baş-son boşlukları kırp
    return t.strip()

df["chunk_text"] = df["chunk_text"].apply(normalize_text)


In [ ]:
def is_fragment(text: str, max_words: int = 40) -> bool:
    words = text.strip().split()
    if len(words) == 0:
        return True  # boş ise zaten çöp
    if len(words) > max_words:
        return False

    first = words[0].lower()
    fragment_starters = [
        "and", "or", "but", "also", "furthermore", "additionally",
        "however", "moreover", "thus", "therefore"
    ]

    # Örn: "and", "or", "but" ile başlıyorsa ve kısa ise muhtemelen üst chunk'ın devamı
    if first in fragment_starters:
        return True

    return False


In [ ]:
df_sorted = df.sort_values(
    ["filename", "section_label", "section_index", "chunk_index"]
).reset_index(drop=True)

merged_rows = []

for _, row in df_sorted.iterrows():
    text = row["chunk_text"]
    key = (row["filename"], row["section_label"], str(row["section_index"]))

    if merged_rows:
        last = merged_rows[-1]
        last_key = (last["filename"], last["section_label"], str(last["section_index"]))

        # Eğer aynı doküman / aynı section içindeysek ve bu satır fragment ise:
        if key == last_key and is_fragment(text):
            last["chunk_text"] = last["chunk_text"].rstrip() + " " + text.lstrip()
            continue  # yeni satır ekleme, sadece birleşmiş oldu

    # aksi halde yeni satır olarak ekle
    merged_rows.append({
        "filename": row["filename"],
        "source_url": row["source_url"],
        "section_label": row["section_label"],
        "section_index": row["section_index"],
        "chunk_index": row["chunk_index"],  # sonradan yeniden yazacağız
        "chunk_text": text
    })

df_merged = pd.DataFrame(merged_rows)
df_merged.shape


(247, 6)

In [ ]:
df_final_rows = []

for (fname, sec_label, sec_idx), g in df_merged.groupby(
    ["filename", "section_label", "section_index"]
):
    g_sorted = g.sort_values("chunk_index")  # eski index'e göre sırala
    for new_i, (_, r) in enumerate(g_sorted.iterrows()):
        df_final_rows.append({
            "filename": fname,
            "source_url": r["source_url"],
            "section_label": sec_label,
            "section_index": sec_idx,
            "chunk_index": new_i,
            "chunk_text": r["chunk_text"]
        })

df_final = pd.DataFrame(df_final_rows)
df_final.shape, df_final.head(5)


((247, 6),
                                          filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 3  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 4  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...      ISO_5.10   
 2  https://hightable.io/iso-27001-annex-a-control...      ISO_5.11   
 3  https://hightable.io/iso-27001-annex-a-control...      ISO_5.12   
 4  https://hightable.io/iso-27001-annex-a-control...      ISO_5.13   
 
    section_index  chunk_index  \
 0           5.10            0   
 1           5.10            0   
 2           5.11            0   
 3           5.12            0   
 4           5.13            0   
 
                             

In [ ]:
def is_too_short(text: str) -> bool:
    words = text.strip().split()
    # 4 kelimeden azsa ve içinde "5.1", "8.16" gibi ISO numaraları yoksa atalım
    if len(words) < 4 and not re.search(r"\d+\.\d+", text):
        return True
    return False

mask_short = df_final["chunk_text"].apply(is_too_short)
df_final = df_final[~mask_short].reset_index(drop=True)

df_final.shape


(247, 6)

In [ ]:
df_final.to_csv("bilgi_guvenligi_chunks_v3_final.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_chunks_v3_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install huggingface_hub -q

In [ ]:
from huggingface_hub import login

login()

In [ ]:
!pip install -q transformers accelerate sentencepiece

import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/gemma-2-2b-it"  # istersen değiştirilebilir
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

url_v2 = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_final.csv"
df = pd.read_csv(url_v2)
df.shape


(247, 6)

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_final.csv"

df_chunks = pd.read_csv(url)
df_chunks.shape
df_chunks.head(3)


,filename,source_url,section_label,section_index,chunk_index,chunk_text
0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.1,5.10,0,5.1 Policies for information security Informat...
1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.10,5.10,0,5.10 Acceptable use of information and other a...
2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.11,5.11,0,5.11 Return of Assets Personnel and other inte...


In [ ]:
def generate_from_model(prompt, max_new_tokens=220, temperature=0.4):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text[len(prompt):].strip()


In [ ]:
simple_template = """
You are an experienced information security specialist.

Based ONLY on the following policy text, generate 1 or 2 clear question–answer pairs.
Requirements:
- Questions must be directly answerable from the text (no guessing, no external knowledge).
- Vary the question types if possible (what / why / how / which / who).
- Answers must be 1–3 full sentences, not just a phrase or list.
- Use neutral, professional language.
- Do NOT mention the words "chunk" or "snippet" or "this text" in the answer.

Return the result in JSON list format exactly like:
[
  {{"question": "...", "answer": "..."}},
  {{"question": "...", "answer": "..."}}
]

Policy text:
\"\"\"
{context}
\"\"\"

JSON:
"""


In [ ]:
import json

def make_simple_qa_for_chunk(row, max_pairs=2):
    context = row["chunk_text"]
    section_label = str(row["section_label"])

    # Çok kısa veya “Policy Sanctions” kısmı ise max 1 QA üret
    if len(context.split()) < 40 or "Sanctions" in section_label:
        local_max_pairs = 1
    else:
        local_max_pairs = max_pairs

    prompt = simple_template.format(context=context.strip())
    raw = generate_from_model(prompt, max_new_tokens=320, temperature=0.4)

    # JSON parse
    try:
        start = raw.find("[")
        end = raw.rfind("]") + 1
        js = raw[start:end]
        data = json.loads(js)
    except Exception:
        return []  # parse edilemezse bu chunk'ı geç

    out = []
    for item in data:
        q = item.get("question", "").strip()
        a = item.get("answer", "").strip()

        # Basit kalite filtreleri
        if not q or not a:
            continue
        if len(a.split()) < 6:  # 6 kelimeden kısa cevapları ele
            continue
        if "not mentioned in the text" in a.lower():
            # simple QA'da "yok" demesini istemiyoruz, bu negative QA konusu
            continue

        out.append({
            "question": q,
            "answer": a,
            "question_type": "simple",
            "source_filenames": row["filename"],
            "source_sections": str(row["section_label"]),
            "source_chunk_ids": str(row["chunk_index"]),
            "is_impossible": False,
        })

        if len(out) >= local_max_pairs:
            break

    return out


In [ ]:
simple_qas_test = []
subset = df_chunks.head(20)

for _, row in subset.iterrows():
    simple_qas_test.extend(make_simple_qa_for_chunk(row))

len(simple_qas_test), simple_qas_test[:5]


(21,
 [{'question': 'Who needs to acknowledge the information security policy?',
   'answer': 'Relevant personnel and relevant interested parties need to acknowledge the information security policy.',
   'question_type': 'simple',
   'source_filenames': 'ISO-27001-Controls-List-Free-Download-pdf-.pdf',
   'source_sections': 'ISO_5.1',
   'source_chunk_ids': '0',
   'is_impossible': False},
  {'question': 'What process is required for the information security policy to be reviewed?',
   'answer': 'The information security policy is reviewed at planned intervals and if significant changes occur.',
   'question_type': 'simple',
   'source_filenames': 'ISO-27001-Controls-List-Free-Download-pdf-.pdf',
   'source_sections': 'ISO_5.1',
   'source_chunk_ids': '0',
   'is_impossible': False},
  {'question': "What is the policy's focus?",
   'answer': 'The policy focuses on establishing rules for the acceptable use of information and other associated assets.',
   'question_type': 'simple',
   's

In [ ]:
simple_qas = []

for idx, row in df_chunks.iterrows():
    qas = make_simple_qa_for_chunk(row)
    simple_qas.extend(qas)

len(simple_qas)


386

In [ ]:
df_simple = pd.DataFrame(simple_qas)
df_simple.insert(0, "question_id", [f"SIMP{str(i).zfill(4)}" for i in range(1, len(df_simple)+1)])

df_simple.shape
df_simple.head(10)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
0,SIMP0001,Who needs to approve the information security ...,Management must approve the information securi...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,False
1,SIMP0002,What is the standard for reviewing the informa...,The information security policy should be revi...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,False
2,SIMP0003,"Who is responsible for identifying, documentin...",The organization is responsible for identifyin...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.10,0,False
3,SIMP0004,Who is responsible for returning assets?,"Personnel and other interested parties, as app...",simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.11,0,False
4,SIMP0005,What are the factors that determine the classi...,The classification of information is determine...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.12,0,False
5,SIMP0006,What is the purpose of the policy?,The policy outlines the need for procedures fo...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.13,0,False
6,SIMP0007,What are the requirements for information tran...,The policy requires information transfer rules...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.14,0,False
7,SIMP0008,Who is responsible for establishing and implem...,Organizations must establish and implement acc...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.15,0,False
8,SIMP0009,What is the focus of Identity Management as de...,Identity Management encompasses the full life ...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.16,0,False
9,SIMP0010,Who is responsible for managing authentication...,The management process is responsible for the ...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.17,0,False


In [ ]:
df_simple.to_csv("bilgi_guvenligi_qa_simple.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_qa_simple.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_simple.sample(10, random_state=42)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
336,SIMP0337,Who is responsible for ensuring the organizati...,The organization is responsible for ensuring t...,simple,SANS_Safeguard_Selection_Management_Policy_Apr...,Safeguards,2,False
307,SIMP0308,What are the potential consequences of non-com...,Non-compliance with this policy may result in ...,simple,SANS_Resilience_Management_Policy_April2025.pdf,Policy Sanctions,0,False
90,SIMP0091,Who is responsible for managing access to sour...,The policy does not specify who is responsible...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_8.4,0,False
265,SIMP0266,What types of network devices are covered by t...,"This policy covers routers, switches, firewall...",simple,SANS_Network_Device_Management_Policy_April202...,Purpose,0,False
150,SIMP0151,What should be the user's perspective on AI-ge...,AI-generated content should be viewed as a sta...,simple,SANS_Artificial_Intelligence_Standard_April202...,full,8,False
297,SIMP0298,What is the primary purpose of PAM systems?,"PAM systems are used to document service, shar...",simple,SANS_Privileged_Account_Management_Policy_Apri...,Safeguards,1,False
78,SIMP0079,What are the requirements for the effective us...,The policy states that rules for the effective...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_8.24,0,False
101,SIMP0102,Who is responsible for ensuring that servers u...,"All servers used for authentication, such as R...",simple,SANS_Acceptable_Encryption_Standard_April2025.pdf,full,2,False
312,SIMP0313,What is the purpose of regularly testing the I...,The policy requires the organization to regula...,simple,SANS_Resilience_Management_Policy_April2025.pdf,Safeguards,2,False
33,SIMP0034,Who should the organization establish contact ...,The organization should establish contact with...,simple,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.5,0,False


In [ ]:
import pandas as pd

df_simple = pd.read_csv("bilgi_guvenligi_qa_simple.csv")
print("Önce:", df_simple.shape)

# Tamamen aynı soru+cevap çiftlerini tekilleştir
df_simple_dedup = df_simple.drop_duplicates(subset=["question", "answer"]).reset_index(drop=True)
print("Sonra (QA bazlı tekilleştirme):", df_simple_dedup.shape)

# İstersen sadece soru bazında da düşürebilirsin (opsiyonel):
# df_simple_dedup = df_simple_dedup.drop_duplicates(subset=["question"]).reset_index(drop=True)

df_simple_dedup.to_csv("bilgi_guvenligi_qa_simple_dedup.csv", index=False)


Önce: (386, 8)
Sonra (QA bazlı tekilleştirme): (383, 8)


In [ ]:
import pandas as pd

chunks_url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_final.csv"
df_chunks = pd.read_csv(chunks_url)

df_chunks.shape
df_chunks.head(3)


,filename,source_url,section_label,section_index,chunk_index,chunk_text
0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.1,5.10,0,5.1 Policies for information security Informat...
1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.10,5.10,0,5.10 Acceptable use of information and other a...
2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.11,5.11,0,5.11 Return of Assets Personnel and other inte...


In [ ]:
!pip install -q transformers accelerate sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/gemma-2-2b-it"  # istersen değiştirilebilir
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

def generate_from_model(prompt, max_new_tokens=260, temperature=0.4):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text[len(prompt):].strip()


In [ ]:
complex_template = """
You are an experienced information security specialist.

You will be given TWO related policy snippets (A and B).
Create ONE question that can ONLY be answered by combining information from BOTH snippets.
Then provide ONE clear, concise answer (2–4 sentences) that uses information from both snippets.

Requirements:
- Question must reference both snippets (directly or indirectly).
- Answer must clearly integrate details from both snippets.
- Do NOT invent facts that are not present in the text.
- Do NOT mention "snippet A/B" or "this text"; write it like a real user Q&A.
- Return ONLY a JSON object in the format:
{{
  "question": "...",
  "answer": "..."
}}

Snippet A:
\"\"\"
{ctx1}
\"\"\"

Snippet B:
\"\"\"
{ctx2}
\"\"\"

JSON:
"""


In [ ]:
import json

def make_complex_qa(row1, row2):
    ctx1 = str(row1["chunk_text"]).strip()
    ctx2 = str(row2["chunk_text"]).strip()

    # Çok kısa ikili ise atla
    if len(ctx1.split()) < 30 or len(ctx2.split()) < 30:
        return None

    prompt = complex_template.format(ctx1=ctx1, ctx2=ctx2)
    raw = generate_from_model(prompt, max_new_tokens=320, temperature=0.45)

    try:
        start = raw.find("{")
        end = raw.rfind("}") + 1
        js = raw[start:end]
        item = json.loads(js)
    except Exception:
        return None

    q = item.get("question", "").strip()
    a = item.get("answer", "").strip()

    # Basit kalite filtresi
    if not q or not a:
        return None
    if len(a.split()) < 10:  # çok kısa cevapları ele
        return None

    return {
        "question": q,
        "answer": a,
        "question_type": "complex",
        "source_filenames": f"{row1['filename']}|{row2['filename']}",
        "source_sections": f"{row1['section_label']}|{row2['section_label']}",
        "source_chunk_ids": f"{row1['chunk_index']}|{row2['chunk_index']}",
        "is_impossible": False,
    }


In [ ]:
complex_qas = []

# Aynı dosya + aynı section_label içinde ardışık chunk'ları eşleştirelim
for (fname, sec_label), g in df_chunks.groupby(["filename", "section_label"]):
    g_sorted = g.sort_values("chunk_index").reset_index(drop=True)
    for i in range(len(g_sorted) - 1):
        row1 = g_sorted.iloc[i]
        row2 = g_sorted.iloc[i+1]
        qa = make_complex_qa(row1, row2)
        if qa:
            complex_qas.append(qa)
        # İstersen her group için limit koy:
        # if len(complex_qas) >= 150:
        #     break
    # if len(complex_qas) >= 150:
    #     break

len(complex_qas)


102

In [ ]:
df_complex = pd.DataFrame(complex_qas)
df_complex.insert(0, "question_id", [f"COMP{str(i).zfill(4)}" for i in range(1, len(df_complex)+1)])

df_complex.shape
df_complex.head(10)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
0,COMP0001,"According to the policy documents, what is the...",The policy strongly recommends the Advanced En...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,0|1,False
1,COMP0002,"According to the policies, what is the recomme...",To authenticate public keys used to establish ...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,1|2,False
2,COMP0003,What are the potential consequences of violati...,Violating the security policy regarding the us...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,2|3,False
3,COMP0004,What are the specific actions employees must t...,Employees must ensure that company-owned devic...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,0|1,False
4,COMP0005,What are the specific security measures employ...,Employees are expected to secure company-issue...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,1|2,False
5,COMP0006,What are some activities that are considered u...,The company prohibits activities that violate ...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,2|3,False
6,COMP0007,"According to the policy documents, what activi...",The policy documents state that accessing data...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,3|4,False
7,COMP0008,What is explicitly prohibited in terms of netw...,The company prohibits network activity that in...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,4|5,False
8,COMP0009,What are some activities that are expressly pr...,The company explicitly prohibits activities th...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,5|6,False
9,COMP0010,What actions are prohibited for employees when...,Employees must be aware that when using compan...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,6|7,False


In [ ]:
df_complex.to_csv("bilgi_guvenligi_qa_complex.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_qa_complex.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_complex.sample(5, random_state=42)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
30,COMP0031,"According to the policy documents, what are th...",The two main ethical principles that must be a...,complex,SANS_Artificial_Intelligence_Standard_April202...,full|full,16|17,False
67,COMP0068,"According to the policy, what security measure...",The policy requires the use of perimeter netwo...,complex,SANS_Perimeter_Network_Access_Management_Polic...,Safeguards|Safeguards,0|1,False
62,COMP0063,What are the potential consequences for violat...,Violating the organization's cybersecurity pol...,complex,SANS_Internal_Network_Access_Management_Policy...,Safeguards|Safeguards,1|2,False
47,COMP0048,What are the specific data management requirem...,The organization's data inventory management s...,complex,SANS_Data_Inventory_Management_Policy_April202...,Safeguards|Safeguards,1|2,False
42,COMP0043,"According to the policies, what two security m...",The policies require the implementation of a D...,complex,SANS_Cloud_Service_Provider_Management_Policy_...,Safeguards|Safeguards,0|1,False


In [ ]:
import pandas as pd

# Chunk veri seti (kaynak metinler)
chunks_url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_final.csv"
df_chunks = pd.read_csv(chunks_url)

# Daha önce ürettiğin COMPLEX QA dosyası
df_complex_old = pd.read_csv("bilgi_guvenligi_qa_complex.csv")  # eğer dosya adın farklıysa burayı değiştir
print("Eski complex QA:", df_complex_old.shape)
df_chunks.shape, df_chunks.head(2), df_complex_old.head(2)


Eski complex QA: (102, 8)


((247, 6),
                                          filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...      ISO_5.10   
 
    section_index  chunk_index  \
 0            5.1            0   
 1            5.1            0   
 
                                           chunk_text  
 0  5.1 Policies for information security Informat...  
 1  5.10 Acceptable use of information and other a...  ,
   question_id                                           question  \
 0    COMP0001  According to the policy documents, what is the...   
 1    COMP0002  According to the policies, what is the recomme...   
 
                                               answer question_type  \
 0  The policy strongly recommends the Advanced En...    

In [ ]:
complex_template = """
You are an experienced information security specialist.

You will be given TWO related policy snippets (A and B).
Create ONE question that can ONLY be answered by combining information from BOTH snippets.
Then provide ONE clear, concise answer (2–4 sentences) that uses information from both snippets.

Requirements:
- The question must logically depend on BOTH snippets.
- The answer must clearly integrate details from BOTH snippets.
- Do NOT invent facts that are not present in the text.
- Do NOT mention "snippet A/B" or "this text"; write it as a normal policy Q&A.
- Return ONLY a JSON object in the format:
{{
  "question": "...",
  "answer": "..."
}}

Snippet A:
\"\"\"
{ctx1}
\"\"\"

Snippet B:
\"\"\"
{ctx2}
\"\"\"

JSON:
"""


In [ ]:
def make_complex_qa(row1, row2):
    ctx1 = str(row1["chunk_text"]).strip()
    ctx2 = str(row2["chunk_text"]).strip()

    # Çok kısa iki metinle uğraşma
    if len(ctx1.split()) < 30 or len(ctx2.split()) < 30:
        return None

    prompt = complex_template.format(ctx1=ctx1, ctx2=ctx2)
    raw = generate_from_model(prompt, max_new_tokens=320, temperature=0.45)

    try:
        start = raw.find("{")
        end = raw.rfind("}") + 1
        js = raw[start:end]
        item = json.loads(js)
    except Exception:
        return None

    q = item.get("question", "").strip()
    a = item.get("answer", "").strip()

    # Basit kalite filtreleri
    if not q or not a:
        return None
    if len(a.split()) < 10:  # çok kısa cevap
        return None
    if "not mentioned in the text" in a.lower():
        # bu tarz cevaplar negative QA'ya daha uygun, burada istemiyoruz
        return None

    return {
        "question": q,
        "answer": a,
        "question_type": "complex",
        "source_filenames": f"{row1['filename']}|{row2['filename']}",
        "source_sections": f"{row1['section_label']}|{row2['section_label']}",
        "source_chunk_ids": f"{row1['chunk_index']}|{row2['chunk_index']}",
        "is_impossible": False,
    }


In [ ]:
new_complex_qas = []

for (fname, sec_label), g in df_chunks.groupby(["filename", "section_label"]):
    g_sorted = g.sort_values("chunk_index").reset_index(drop=True)

    # İstersen her bölüm için limit koy (çok şişmesin):
    local_count = 0
    for i in range(len(g_sorted) - 1):
        row1 = g_sorted.iloc[i]
        row2 = g_sorted.iloc[i+1]
        qa = make_complex_qa(row1, row2)
        if qa:
            new_complex_qas.append(qa)
            local_count += 1
        if local_count >= 3:  # her (filename, section_label) çifti için max 3 complex QA
            break

len(new_complex_qas)


69

In [ ]:
df_complex_new = pd.DataFrame(new_complex_qas)
print("Yeni üretilen complex QA:", df_complex_new.shape)

# Eski + yeni birleştir
df_complex_all = pd.concat([df_complex_old, df_complex_new], ignore_index=True)
print("Birleştirilmiş (ham):", df_complex_all.shape)

# Soru + cevap bazında tekilleştir
df_complex_all = df_complex_all.drop_duplicates(subset=["question", "answer"]).reset_index(drop=True)
print("Tekilleştirilmiş (QA bazlı):", df_complex_all.shape)


Yeni üretilen complex QA: (69, 7)
Birleştirilmiş (ham): (171, 8)
Tekilleştirilmiş (QA bazlı): (171, 8)


In [ ]:
if "question_id" in df_complex_all.columns:
    df_complex_all = df_complex_all.drop(columns=["question_id"])

df_complex_all.insert(
    0,
    "question_id",
    [f"COMP{str(i).zfill(4)}" for i in range(1, len(df_complex_all)+1)]
)

df_complex_all.head()



,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
0,COMP0001,"According to the policy documents, what is the...",The policy strongly recommends the Advanced En...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,0|1,False
1,COMP0002,"According to the policies, what is the recomme...",To authenticate public keys used to establish ...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,1|2,False
2,COMP0003,What are the potential consequences of violati...,Violating the security policy regarding the us...,complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,2|3,False
3,COMP0004,What are the specific actions employees must t...,Employees must ensure that company-owned devic...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,0|1,False
4,COMP0005,What are the specific security measures employ...,Employees are expected to secure company-issue...,complex,SANS_Acceptable_Use_Standard_April2025.pdf|SAN...,Safeguards|Safeguards,1|2,False


In [ ]:
df_complex_all.to_csv("bilgi_guvenligi_qa_complex_final.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_qa_complex_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_complex_all.sample(10, random_state=42)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
101,COMP0102,What are the potential consequences of failing...,Failure to comply with the Vulnerability Manag...,complex,SANS_Vulnerability_Management_Policy_April2025...,Safeguards|Safeguards,1|2,False
55,COMP0056,What are the key security measures outlined in...,The policy outlines several security measures ...,complex,SANS_Email_Management_Policy_April2025.pdf|SAN...,Safeguards|Safeguards,0|1,False
56,COMP0057,"According to the policy, what are the potentia...",Non-compliance with the email security policie...,complex,SANS_Email_Management_Policy_April2025.pdf|SAN...,Safeguards|Safeguards,1|2,False
139,COMP0140,What safeguards are included in the organizati...,The organization's documented physical securit...,complex,SANS_Physical_Security_Management_Policy_April...,Safeguards|Safeguards,0|1,False
157,COMP0158,"According to the policy, which central softwar...",The policy states that the organization must d...,complex,SANS_Safeguard_Validation_Management_Policy_Ap...,Safeguards|Safeguards,1|2,False
78,COMP0079,"According to the policy, what specific process...",The policy outlines the need to define a proce...,complex,SANS_Resilience_Management_Policy_April2025.pd...,Safeguards|Safeguards,0|1,False
135,COMP0136,What network management system requirements ar...,The organization's network device management s...,complex,SANS_Network_Device_Management_Policy_April202...,Safeguards|Safeguards,0|1,False
104,COMP0105,What are the potential consequences of violati...,"Violations of the key generation policy, such ...",complex,SANS_Acceptable_Encryption_Standard_April2025....,full|full,2|3,False
109,COMP0110,What are the potential consequences for non-co...,Non-compliance with the Access Control List (A...,complex,SANS_Access_Management_Policy_April2025.pdf|SA...,Safeguards|Safeguards,1|2,False
108,COMP0109,"Based on the provided policies, what is the mi...","To access code repositories, a member of the w...",complex,SANS_Access_Management_Policy_April2025.pdf|SA...,Safeguards|Safeguards,0|1,False


In [ ]:
import pandas as pd

chunks_url = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_final.csv"
df_chunks = pd.read_csv(chunks_url)
df_chunks.shape, df_chunks.head(3)


((247, 6),
                                          filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...      ISO_5.10   
 2  https://hightable.io/iso-27001-annex-a-control...      ISO_5.11   
 
    section_index  chunk_index  \
 0           5.10            0   
 1           5.10            0   
 2           5.11            0   
 
                                           chunk_text  
 0  5.1 Policies for information security Informat...  
 1  5.10 Acceptable use of information and other a...  
 2  5.11 Return of Assets Personnel and other inte...  )

In [ ]:
negative_template = """
You are an experienced information security specialist.

You will be given a policy snippet.
Your task is to create ONE question that a user might realistically ask about information security
which CANNOT be answered from this text alone (the information is missing or out of scope).
Then provide ONE short answer (1–3 sentences) that explicitly states that the information is not
specified in the policy text.

Requirements:
- The question should be related to information security or policy usage, but the answer MUST NOT
  be present in the text.
- The answer MUST clearly say that this information is not specified in the policy snippet.
- Do NOT invent details or make assumptions beyond the text.
- Return ONLY a JSON object in the format:
{{
  "question": "...",
  "answer": "..."
}}

Policy snippet:
\"\"\"
{context}
\"\"\"

JSON:
"""


In [ ]:
def make_negative_qa(row):
    context = str(row["chunk_text"]).strip()
    if len(context.split()) < 40:
        # çok kısa metinlerden negative QA üretmeye çalışma, saçmalama riski yüksek
        return None

    prompt = negative_template.format(context=context)
    raw = generate_from_model(prompt, max_new_tokens=260, temperature=0.45)

    try:
        start = raw.find("{")
        end = raw.rfind("}") + 1
        js = raw[start:end]
        item = json.loads(js)
    except Exception:
        return None

    q = item.get("question", "").strip()
    a = item.get("answer", "").strip()

    if not q or not a:
        return None
    if len(a.split()) < 8:
        return None
    # Güvenlik: metinde geçen şeyi yanlışlıkla "yok" demesin diye basit check
    snippet_lower = context.lower()
    if any(word in snippet_lower for word in ["password", "encryption", "firewall", "access control"]):
        # Eğer cevapta da bunlar geçiyorsa şüphelenebilirsin, manuel göz atılabilir.
        pass

    return {
        "question": q,
        "answer": a,
        "question_type": "negative",
        "source_filenames": row["filename"],
        "source_sections": str(row["section_label"]),
        "source_chunk_ids": str(row["chunk_index"]),
        "is_impossible": True,
    }


In [ ]:
import numpy as np

negative_qas = []

# Karma bir örnek için rasgele chunk seçelim (ör: 120 chunk -> her 2 taneden 1'i başarıyla gelirse ~50-60 kayıt)
sample_indices = np.random.choice(len(df_chunks), size=120, replace=False)
sampled_chunks = df_chunks.iloc[sample_indices]

for _, row in sampled_chunks.iterrows():
    qa = make_negative_qa(row)
    if qa:
        negative_qas.append(qa)
    if len(negative_qas) >= 60:
        break

len(negative_qas)


60

In [ ]:
df_negative = pd.DataFrame(negative_qas)
df_negative.insert(0, "question_id", [f"NEG{str(i).zfill(4)}" for i in range(1, len(df_negative)+1)])

df_negative.shape
df_negative.head(10)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
0,NEG0001,What are the specific requirements for ensurin...,This policy snippet does not specify the requi...,negative,SANS_Mobile_Device_Management_Policy_April2025...,Safeguards,1,True
1,NEG0002,How often should the organization conduct cybe...,This policy snippet does not specify the frequ...,negative,SANS_Safeguard_Validation_Management_Policy_Ap...,Safeguards,2,True
2,NEG0003,What specific protocols or technologies must b...,This policy snippet does not specify the speci...,negative,SANS_Perimeter_Network_Access_Management_Polic...,Safeguards,1,True
3,NEG0004,What are the specific requirements for user pe...,The provided policy snippet does not specify t...,negative,SANS_System_Protection_Management_Policy_April...,Safeguards,0,True
4,NEG0005,What are the specific security requirements fo...,This policy snippet does not specify the secur...,negative,SANS_Artificial_Intelligence_Standard_April202...,full,2,True
5,NEG0006,What are the security implications of storing ...,This information is not specified in the polic...,negative,SANS_Database_Credentials_Standard_April2025.pdf,full,3,True
6,NEG0007,How are public keys used for authentication an...,This policy snippet does not specify the recom...,negative,SANS_Acceptable_Encryption_Standard_April2025.pdf,full,1,True
7,NEG0008,What specific types of legal consequences coul...,The policy does not specify the specific types...,negative,SANS_Email_Management_Policy_April2025.pdf,Safeguards,2,True
8,NEG0009,What are the specific requirements for accessi...,The policy text does not specify the specific ...,negative,SANS_Artificial_Intelligence_Standard_April202...,full,22,True
9,NEG0010,What are the specific types of alerts that nee...,This policy does not specify the types of aler...,negative,SANS_Log_Management_Policy_April2025.pdf,Safeguards,2,True


In [ ]:
df_negative.to_csv("bilgi_guvenligi_qa_negative.csv", index=False)


In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_qa_negative.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_negative.sample(8, random_state=42)


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible
0,NEG0001,What are the specific requirements for ensurin...,This policy snippet does not specify the requi...,negative,SANS_Mobile_Device_Management_Policy_April2025...,Safeguards,1,True
5,NEG0006,What are the security implications of storing ...,This information is not specified in the polic...,negative,SANS_Database_Credentials_Standard_April2025.pdf,full,3,True
36,NEG0037,What specific cybersecurity safeguards are req...,This policy snippet does not specify the speci...,negative,SANS_Safeguard_Selection_Management_Policy_Apr...,Safeguards,0,True
45,NEG0046,What specific cybersecurity safeguards are out...,This policy snippet does not specify the exact...,negative,SANS_Risk_Communication_Management_Policy_Apri...,Safeguards,0,True
13,NEG0014,What specific procedures are outlined for inci...,This policy snippet does not specify procedure...,negative,SANS_Physical_Security_Management_Policy_April...,Safeguards,0,True
54,NEG0055,What are the specific requirements for data re...,This policy snippet does not specify requireme...,negative,SANS_Privacy_Management_Policy_April2025.pdf,Safeguards,1,True
33,NEG0034,What are the specific procedures for conductin...,The policy snippet does not specify the proced...,negative,SANS_Safeguard_Validation_Management_Policy_Ap...,Safeguards,1,True
48,NEG0049,What specific legal or regulatory framework mu...,The policy text does not specify the specific ...,negative,SANS_Artificial_Intelligence_Standard_April202...,full,17,True


In [ ]:
import pandas as pd

# Eğer dosyaları Colab'a yüklediysen:
df_simple  = pd.read_csv("bilgi_guvenligi_qa_simple_dedup.csv")
df_complex = pd.read_csv("bilgi_guvenligi_qa_complex_final.csv")
df_negative = pd.read_csv("bilgi_guvenligi_qa_negative.csv")

print("Simple:", df_simple.shape)
print("Complex:", df_complex.shape)
print("Negative:", df_negative.shape)


Simple: (383, 8)
Complex: (171, 8)
Negative: (60, 8)


In [ ]:
cols = [
    "question_id",
    "question",
    "answer",
    "question_type",
    "source_filenames",
    "source_sections",
    "source_chunk_ids",
    "is_impossible"
]

df_simple   = df_simple.reindex(columns=cols)
df_complex  = df_complex.reindex(columns=cols)
df_negative = df_negative.reindex(columns=cols)

df_simple.head(2), df_complex.head(2), df_negative.head(2)


(  question_id                                           question  \
 0    SIMP0001  Who needs to approve the information security ...   
 1    SIMP0002  What is the standard for reviewing the informa...   
 
                                               answer question_type  \
 0  Management must approve the information securi...        simple   
 1  The information security policy should be revi...        simple   
 
                                  source_filenames source_sections  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf         ISO_5.1   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf         ISO_5.1   
 
    source_chunk_ids  is_impossible  
 0                 0          False  
 1                 0          False  ,
   question_id                                           question  \
 0    COMP0001  According to the policy documents, what is the...   
 1    COMP0002  According to the policies, what is the recomme...   
 
                                        

In [ ]:
df_all = pd.concat([df_simple, df_complex, df_negative], ignore_index=True)

print("Toplam QA satırı:", df_all.shape)
df_all["question_type"].value_counts()


Toplam QA satırı: (614, 8)


,count
question_type,
simple,383
complex,171
negative,60


In [ ]:
df_all["question"] = df_all["question"].astype(str).str.strip()
df_all["answer"] = df_all["answer"].astype(str).str.strip()

df_all = df_all[(df_all["question"] != "") & (df_all["answer"] != "")]
df_all.shape


(614, 8)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df_all,
    test_size=0.15,
    random_state=42,
    stratify=df_all["question_type"]  # simple/complex/negative oranını korur
)

train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

train_df["dataset_split"] = "train"
test_df["dataset_split"] = "test"

df_all_split = pd.concat([train_df, test_df], ignore_index=True)
df_all_split.shape, df_all_split["dataset_split"].value_counts()


((614, 9),
 dataset_split
 train    521
 test      93
 Name: count, dtype: int64)

In [ ]:
df_all_split = df_all_split.drop(columns=["question_id"], errors="ignore")
df_all_split.insert(0, "question_id", [f"QA{str(i).zfill(4)}" for i in range(1, len(df_all_split)+1)])


In [ ]:
df_all_split.to_csv("bilgi_guvenligi_qa_master.csv", index=False)

from google.colab import files
files.download("bilgi_guvenligi_qa_master.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

def row_to_jsonl_obj(row):
    instr = row["question"]
    out = row["answer"]

    # İstersen soru tipine göre ufak bir prepend ekleyebilirsin (isteğe bağlı):
    # if row["question_type"] == "complex":
    #     instr = "[COMPLEX POLICY QUESTION]\n" + instr
    # elif row["question_type"] == "negative":
    #     instr = "[IMPOSSIBLE / NOT IN POLICY]\n" + instr

    return {
        "instruction": instr,
        "input": "",
        "output": out
    }

train_jsonl_path = "bilgi_guvenligi_finetune_train.jsonl"
test_jsonl_path  = "bilgi_guvenligi_finetune_test.jsonl"

with open(train_jsonl_path, "w", encoding="utf-8") as f:
    for _, row in train_df.iterrows():
        obj = row_to_jsonl_obj(row)
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

with open(test_jsonl_path, "w", encoding="utf-8") as f:
    for _, row in test_df.iterrows():
        obj = row_to_jsonl_obj(row)
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

train_jsonl_path, test_jsonl_path


('bilgi_guvenligi_finetune_train.jsonl', 'bilgi_guvenligi_finetune_test.jsonl')

In [ ]:
from google.colab import files
files.download("bilgi_guvenligi_finetune_train.jsonl")
files.download("bilgi_guvenligi_finetune_test.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>